In [1]:
%matplotlib inline 
import matplotlib.pylab as plt 
import numpy as np 
import pandas as pd 
import seaborn as sns 
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('./datasets/Uganda_seasonal.csv')
data.head()

,District,year,Season,EVI_lag1,EVI_lag2,EVI_lag3,NDVI_lag1,NDVI_lag2,NDVI_lag3,SPEI_10month_lag1,...,surface_temperature_daytime_lag3,surface_temperature_nighttime_lag1,surface_temperature_nighttime_lag2,surface_temperature_nighttime_lag3,wind_speed_lag1,wind_speed_lag2,wind_speed_lag3,drought_reported,drought_news_article,drought_desinventar
0,ABIM,2000,6_7,0.365380,0.184616,0.137156,0.624764,0.368438,0.270712,NaN,...,315.658256,292.344175,293.315996,294.617049,3.503192,4.994076,6.794279,0,0,0
1,ABIM,2000,11_12,0.356362,0.398800,0.456916,0.594449,0.621502,0.700513,NaN,...,300.157214,290.764633,290.519499,290.596247,3.927143,3.695719,2.923477,0,0,0
2,ABIM,2001,6_7,0.441114,0.334390,0.158305,0.705884,0.590362,0.294246,0.072971,...,315.265358,291.998724,292.858063,294.444529,3.024279,5.153910,6.539992,0,0,0
3,ABIM,2001,11_12,0.327405,0.360157,0.430884,0.554036,0.622010,0.690541,1.184276,...,300.042366,290.867492,290.913252,291.171227,3.747200,3.007668,3.130027,0,0,0
4,ABIM,2002,6_7,0.414888,0.227817,0.177122,0.661810,0.465133,0.365969,-1.047781,...,310.920768,292.783299,293.772280,293.698790,3.700571,6.761159,5.896890,0,0,0


In [3]:
All_label_list = ['drought_reported', 'drought_news_article', 'drought_desinventar']

label_name = 'drought_reported'

All_feature_list = list(data.drop(labels=['District','year','Season']+All_label_list, 
                                  axis=1).columns)
SPEI_features = list(np.array(All_feature_list)[list(map(lambda x: x.split('_')[0]=='SPEI',All_feature_list))])

feature_list_No_SPEI = list(data.drop(labels=['District','year','Season']+All_label_list+SPEI_features, 
                                  axis=1).columns)

In [4]:
mapping ={}
for col in feature_list_No_SPEI:
    mapping[col]=col.split('_lag')[0]


In [5]:
mean_features = []
for name,group in data.groupby(by=mapping,axis = 1):
    data[name+'_mean']=group.mean(axis=1)
    All_feature_list += [name+'_mean']
    feature_list_No_SPEI += [name+'_mean']
    mean_features +=[name+'_mean']
mean_features +=['SPEI_3month_lag1']

In [6]:
data.head()

,District,year,Season,EVI_lag1,EVI_lag2,EVI_lag3,NDVI_lag1,NDVI_lag2,NDVI_lag3,SPEI_10month_lag1,...,SoilTemperature10_40cm_mean,SoilTemperature40_100cm_mean,air_temperature_mean,evapotranspiration_mean,precipitation_per_hour_v1_mean,precipitation_per_hour_v2_mean,rainfall_mean,surface_temperature_daytime_mean,surface_temperature_nighttime_mean,wind_speed_mean
0,ABIM,2000,6_7,0.365380,0.184616,0.137156,0.624764,0.368438,0.270712,NaN,...,299.858673,299.736331,299.039990,0.000034,0.130057,63.441352,0.000042,310.413322,293.425740,5.097182
1,ABIM,2000,11_12,0.356362,0.398800,0.456916,0.594449,0.621502,0.700513,NaN,...,295.811595,295.617852,296.293768,0.000053,0.221155,72.217721,0.000059,301.600701,290.626793,3.515447
2,ABIM,2001,6_7,0.441114,0.334390,0.158305,0.705884,0.590362,0.294246,0.072971,...,298.785551,298.866327,298.361880,0.000044,0.199700,111.769755,0.000052,307.870124,293.100439,4.906060
3,ABIM,2001,11_12,0.327405,0.360157,0.430884,0.554036,0.622010,0.690541,1.184276,...,295.937655,295.639653,296.381364,0.000048,0.204175,76.189088,0.000056,301.415378,290.983990,3.294965
4,ABIM,2002,6_7,0.414888,0.227817,0.177122,0.661810,0.465133,0.365969,-1.047781,...,299.175406,299.100020,298.171585,0.000038,0.137777,79.411844,0.000035,308.790876,293.418123,5.452873


In [35]:
grouped = data.groupby(['District','Season'])

normed_data = pd.DataFrame()

Znorm = StandardScaler()

for name,group in grouped:
    group.reset_index(inplace=True,drop=True)
    temp = pd.DataFrame(Znorm.fit_transform(group[feature_list_No_SPEI]), columns=feature_list_No_SPEI)
    temp[['District','year','Season']+All_label_list] = group[['District','year','Season']+All_label_list]
    temp = temp[['District','year','Season']+feature_list_No_SPEI+All_label_list]
    normed_data = pd.concat([normed_data, temp])
normed_data.reset_index(inplace=True,drop=True)

In [36]:
normed_data = normed_data.merge(data[['District','year','Season', 'SPEI_3month_lag1']], 
                  on=['District','year','Season'])
normed_data = normed_data[['District','year','Season']+feature_list_No_SPEI+['SPEI_3month_lag1']+All_label_list]

In [37]:
normed_data.head()

,District,year,Season,EVI_lag1,EVI_lag2,EVI_lag3,NDVI_lag1,NDVI_lag2,NDVI_lag3,SoilMoisture00_10cm_lag1,...,precipitation_per_hour_v1_mean,precipitation_per_hour_v2_mean,rainfall_mean,surface_temperature_daytime_mean,surface_temperature_nighttime_mean,wind_speed_mean,SPEI_3month_lag1,drought_reported,drought_news_article,drought_desinventar
0,ABIM,2000,11_12,0.829612,0.954151,0.874859,0.661969,0.091475,0.348946,1.653157,...,1.601028,-0.469460,1.336204,-0.151764,-1.343323,-0.412148,0.946693,0,0,0
1,ABIM,2001,11_12,0.151990,-0.233896,0.171060,-0.083043,0.103206,0.057286,0.716883,...,0.929294,-0.254581,0.823853,-0.334808,-0.132387,-1.004612,1.014671,0,0,0
2,ABIM,2002,11_12,-1.552285,-2.038942,-0.783698,-1.152559,-2.594844,-1.373710,-0.382440,...,0.584803,-1.405233,-0.482233,2.041320,0.780647,0.752253,0.714817,0,0,0
3,ABIM,2003,11_12,-0.165715,0.721038,0.957039,-0.060101,0.818002,0.764914,-1.305671,...,-1.604576,-1.661421,-1.335585,0.161190,0.848854,1.321092,-1.517769,0,0,0
4,ABIM,2004,11_12,-1.660662,-0.801463,-0.243357,-2.294412,-1.112314,-0.628375,-1.503838,...,-1.021116,-1.550847,-0.415216,0.949508,-0.042240,0.929712,0.386794,0,0,0


In [38]:
normed_data.to_csv('./datasets/Uganda_seasonal_normalized.csv',index=False)

In [39]:
normed_data[['District','year','Season']+mean_features+All_label_list].to_csv('./datasets/Uganda_seasonal_normalized_mean.csv',index=False)